# Compound Events

In [1]:
%%time
import os
import pandas as pd
import warnings
from plot import Plot
from process import Compound
warnings.filterwarnings('ignore')

plotly = False
event = ['CWHE','CDHE', 'CFE'][2]
months = [6, 7, 8]
# months = None
center='LARC'

freq, scale, gamma_n = 'D', 30, [10, 20, 30][2]

metrics = {
    '_duration_mean$':  f'Average {event} Duration',
    '_sequence_total$': f'Total {event} Sequences',
    '_duration_max$':   f'Max {event} Duration',
    '_event_total$':    f'Total {event}',
    '_day_total$':      f'Total {event} Days'
}

def setup_thresholds(event):
    '''Set up thresholds for different severity levels of compound events.'''
    def f_to_c(f):
        return (f - 32) * 5/9
            
    if event == 'CDHE':
        spi_op, tm_op = '<', '>' # spi (standardized), tm (f->c)
        return[{'spi': (spi_op, -1), 'tasmax': (tm_op, f_to_c(90))},
               {'spi': (spi_op, -2), 'tasmax': (tm_op, f_to_c(90))}]
    elif event == 'CWHE':
        spi_op, tm_op = '>', '>' # spi (standardized), tm (f->c)
        return[{'spi': (spi_op, 1), 'tasmax': (tm_op, f_to_c(90))},
               {'spi': (spi_op, 2), 'tasmax': (tm_op, f_to_c(90))}]
    elif event == 'CFE':
        pr_op, rzsm_op = '>', '>' 
        rzsm_thres = [.7, .8]#, .9, .95, .98] # percentiles
        prec_thres = [100, 125]#, 150] # mm/day
        return [{'pr': (pr_op, prec), 'rzsm': (rzsm_op, rzsm)}
                for prec in prec_thres for rzsm in rzsm_thres]
    else:
        raise ValueError('Invalid event type')
        
thresholds = setup_thresholds(event)

# result, compound, groups = Compound(center, event, months, freq, scale, thresholds, gamma_n).main()
pr, rzsm = Compound(center, event, months, freq, scale, thresholds, gamma_n).main()

CPU times: user 982 ms, sys: 189 ms, total: 1.17 s
Wall time: 1.21 s


In [2]:
pr

INM-CM4-8_pr  INM-CM5-0_pr  NorESM2-MM_pr  NorESM2-LM_pr  \
ssp    date                                                                   
ssp126 1950-01-01     30.614947      0.000000       8.900920       0.188122   
       1950-01-02      0.000000      0.000000       9.900842       0.000000   
       1950-01-03      0.000000      0.000000       0.000000       0.000410   
       1950-01-04      0.107407      0.000000       0.060549       2.545188   
       1950-01-05      0.020872      1.650145       0.012392       2.488102   
...                         ...           ...            ...            ...   
ssp370 2100-12-27      0.000685      0.504589       0.000000       0.008440   
       2100-12-28      0.000000      0.273478       0.000000       0.017276   
       2100-12-29      0.000000      1.175679       0.000000      21.858791   
       2100-12-30      0.261979     26.761998       0.000000       4.189230   
       2100-12-31     35.381902      0.824043       0.000000       0.000000   

                   MIROC6_pr  GFDL-ESM4_pr  MIROC-ES2L_pr  GISS-E2-1-G_pr  \
ssp    date                                                                 
ssp126 1950-01-01   0.000000     18.900750       1.798117        6.346050   
       1950-01-02   6.034051      0.371291       0.965638       34.234164   
       1950-01-03   0.000000      0.000000      12.954931        0.000345   
       1950-01-04   0.222752      0.000000       3.076136        0.000909   
       1950-01-05   1.631401      3.111130       0.306291        0.663034   
...                      ...           ...            ...             ...   
ssp370 2100-12-27   0.000000      0.000000       0.008248        0.000419   
       2100-12-28   0.015452      0.000000       1.856367        0.073050   
       2100-12-29   1.804309     16.479281       6.132260        3.648153   
       2100-12-30   1.827435     21.489239       2.076938        0.000000   
       2100-12-31   0.000000      0.115352       0.578609       40.011048   

                   FGOALS-g3_pr  MPI-ESM1-2-HR_pr  ...  MRI-ESM2-0_pr  \
ssp    date                                        ...                  
ssp126 1950-01-01      1.274339          0.000000  ...       0.000000   
       1950-01-02      0.226831          0.071077  ...      25.434093   
       1950-01-03      2.854043          8.961192  ...      13.961951   
       1950-01-04      5.153899          1.016091  ...       0.000325   
       1950-01-05      0.390431          0.283590  ...       0.125417   
...                         ...               ...  ...            ...   
ssp370 2100-12-27      0.002800          0.000000  ...       0.000260   
       2100-12-28      0.000312          0.000000  ...       0.263888   
       2100-12-29      2.677424          0.000000  ...       1.275888   
       2100-12-30      6.199204          0.082812  ...       0.115360   
       2100-12-31      7.893869         29.217961  ...       0.000000   

                   CMCC-ESM2_pr  ACCESS-ESM1-5_pr  EC-Earth3_pr  \
ssp    date                                                       
ssp126 1950-01-01      5.724272          0.514907      0.000000   
       1950-01-02      3.482699         10.733470      0.000000   
       1950-01-03      2.021233          0.003342      0.000000   
       1950-01-04      0.000000          0.000000      0.000000   
       1950-01-05      0.000000          0.000000      0.000000   
...                         ...               ...           ...   
ssp370 2100-12-27      0.214784          0.000000      0.000000   
       2100-12-28      0.000000          0.000000      0.000000   
       2100-12-29      0.000000          0.004435      0.000000   
       2100-12-30      0.000000          0.002551      6.921214   
       2100-12-31     50.182629          0.000000      0.000000   

                   ACCESS-CM2_pr  IPSL-CM6A-LR_pr  CNRM-ESM2-1_pr  \
ssp    date                                                         
ssp126 1950-01-01       0.000

In [3]:
# 1950-2100
pr.describe(percentiles=[0.995, 0.999, 0.9999]).T

,count,mean,std,min,50%,99.5%,99.9%,99.99%,max
INM-CM4-8_pr,165456.0,3.344395,6.206291,0.0,0.420654,35.705897,53.408559,79.242462,99.234726
INM-CM5-0_pr,165456.0,3.441820,6.285606,0.0,0.502541,36.017580,53.937049,78.971958,104.136711
NorESM2-MM_pr,165456.0,3.325623,6.905962,0.0,0.066280,40.530824,61.254389,92.243242,110.459560
NorESM2-LM_pr,165456.0,3.344801,6.365872,0.0,0.472813,37.149486,56.253027,84.565723,94.338645
MIROC6_pr,165456.0,3.424175,7.031797,0.0,0.121038,41.165998,62.312368,94.763011,122.226935
GFDL-ESM4_pr,165456.0,3.407082,6.812967,0.0,0.195902,39.547057,55.967543,86.910200,112.713169
MIROC-ES2L_pr,165456.0,3.483730,6.229181,0.0,0.747435,37.031987,52.519959,79.013030,86.851509
GISS-E2-1-G_pr,165456.0,3.475920,6.943196,0.0,0.329017,41.696433,62.508466,90.392676,100.579418
FGOALS-g3_pr,165456.0,3.368398,6.129397,0.0,0.615638,34.899430,49.647192,76.351659,87.711172
MPI-ESM1-2-HR_pr,165456.0,3.368778,6.754771,0.0,0.233755,39.910709,59.533502,91.121098,128.901776


In [4]:
# 1950-2014
dates = pr.index.get_level_values('date')
pr[(1950 <= dates.year) & (dates.year <= 2014)].describe(percentiles=[0.995, 0.999, 0.9999]).T

,count,mean,std,min,50%,99.5%,99.9%,99.99%,max
INM-CM4-8_pr,71223.0,3.303050,6.161168,0.0,0.403412,35.475662,52.286550,86.101766,99.234726
INM-CM5-0_pr,71223.0,3.258167,6.051570,0.0,0.399555,33.846757,51.684706,75.003043,94.925245
NorESM2-MM_pr,71223.0,3.302059,6.837183,0.0,0.064709,39.419345,60.794582,95.873037,101.870489
NorESM2-LM_pr,71223.0,3.305878,6.187710,0.0,0.463080,35.526044,50.929922,85.032862,86.123230
MIROC6_pr,71223.0,3.280189,6.730313,0.0,0.105050,37.392377,58.841243,97.069251,122.226935
GFDL-ESM4_pr,71223.0,3.281642,6.535805,0.0,0.168516,36.565340,54.469191,86.910200,106.235272
MIROC-ES2L_pr,71223.0,3.337487,5.934497,0.0,0.725651,34.507936,50.651543,80.498089,86.851509
GISS-E2-1-G_pr,71223.0,3.306807,6.490273,0.0,0.315179,37.527435,54.029285,90.921504,100.579418
FGOALS-g3_pr,71223.0,3.288887,5.930371,0.0,0.604476,33.682616,46.435994,74.157162,82.251490
MPI-ESM1-2-HR_pr,71223.0,3.300775,6.587864,0.0,0.241312,38.006435,54.839054,103.334806,128.901776


In [5]:
# 2015-2100
dates = pr.index.get_level_values('date')
pr[(2015 <= dates.year) & (dates.year <= 2100)].describe(percentiles=[0.995, 0.999, 0.9999]).T

,count,mean,std,min,50%,99.5%,99.9%,99.99%,max
INM-CM4-8_pr,94233.0,3.375643,6.240031,0.0,0.436710,35.943165,53.999979,76.605929,90.477562
INM-CM5-0_pr,94233.0,3.580628,6.453433,0.0,0.594547,37.245108,55.651207,79.977112,104.136711
NorESM2-MM_pr,94233.0,3.343433,6.957479,0.0,0.067492,41.949141,61.845238,91.314563,110.459560
NorESM2-LM_pr,94233.0,3.374220,6.497167,0.0,0.480330,38.782166,58.199677,81.639987,94.338645
MIROC6_pr,94233.0,3.533003,7.249489,0.0,0.135899,42.850933,64.459888,90.864752,121.760980
GFDL-ESM4_pr,94233.0,3.501893,7.013734,0.0,0.220600,41.224575,57.986334,84.771392,112.713169
MIROC-ES2L_pr,94233.0,3.594263,6.440799,0.0,0.765059,38.372664,53.319960,74.377050,86.302204
GISS-E2-1-G_pr,94233.0,3.603738,7.264238,0.0,0.339710,44.186426,65.159794,88.828302,97.307911
FGOALS-g3_pr,94233.0,3.428494,6.275000,0.0,0.625637,35.909198,53.061678,76.613869,87.711172
MPI-ESM1-2-HR_pr,94233.0,3.420176,6.877824,0.0,0.229514,40.838301,61.126149,88.526564,101.818253
